# Porównania cen

In [1]:
import pandas as pd
import re
from matplotlib.ticker import MultipleLocator
%pylab inline 
%matplotlib widget

Populating the interactive namespace from numpy and matplotlib


In [55]:
df_good = pd.DataFrame(pd.read_json("../Data/Full-URLs-good-domain-only.json")["domain"]).reset_index(drop=True)
df_bad = pd.DataFrame(pd.read_json("../Data/Full-URLs-bad-domain-only.json")["domain"]).reset_index(drop=True)

In [56]:
df_prices = pd.read_csv("../Data/domains-prices.csv", index_col=0)

In [58]:
df_good = df_good.drop(705902, axis=0)
df_good["domain"].iloc[705900:705910]

705900        www.bip.pm155lodz.wikom.pl
705901     http://pm155lodz.bip.wikom.pl
705903                pm156lodz.wikom.pl
705904    https://pm156lodz.bip.wikom.pl
705905        www.bip.pm156lodz.wikom.pl
705906                pm159lodz.wikom.pl
705907        www.bip.pm159lodz.wikom.pl
705908    https://pm159lodz.bip.wikom.pl
705909       http://pm16.bip.gliwice.eu/
705910             www.p16.edu.gorzow.pl
Name: domain, dtype: object

In [59]:
def parse_url(x):
    regex = re.match("^((?P<scheme>[^:/?#]+):(?=//))?(//)?(((?P<login>[^:]+)" + 
                     "(?::(?P<password>[^@]+)?)?@)?(?P<host>[^@/?#:]*)" + 
                     "(?::(?P<port>\d+)?)?)?(?P<path>[^?#]*)(\?(?P<query>[^#]*))?(#(?P<fragment>.*))?", x)
    return regex.group('host').split('.')[-1]

In [60]:
df_good["tld"] = df_good["domain"].apply(parse_url)
df_bad["tld"] = df_bad["domain"].apply(parse_url)

In [61]:
df_good.head()

,domain,tld
0,google.com,com
1,youtube.com,com
2,tmall.com,com
3,baidu.com,com
4,qq.com,com


In [62]:
df_bad.head()

,domain,tld
0,https://www.adonapuppieshome.com,com
1,https://www.safetimex.com,com
2,http://www.royal-australian-air-force-gov-au.com,com
3,http://www.airforce-raaf.org,org
4,https://www.arabellapuppies.com,com


In [63]:
df_prices["tld"] = df_prices["TLD"].str.split(pat=".", expand=True)[1]
df_prices.head()

,Registration Price,Renewal Price,TLD,Minimum Term,tld
0,59.99,59.99,.ac,1 Year,ac
1,13.99,33.99,.academy,1 Year,academy
2,19.99,87.99,.accountant,1 Year,accountant
3,16.99,108.99,.accountants,1 Year,accountants
4,13.99,42.99,.actor,1 Year,actor


In [64]:
df_tld_good_res = pd.DataFrame(df_good.groupby("tld").count().sort_values("domain", ascending=False))
df_tld_good_res = df_tld_good_res.reset_index()
df_tld_good_res["label"] = "good"
df_tld_good_res.head()

,tld,domain,label
0,com,1604974,good
1,org,265271,good
2,net,115300,good
3,uk,97508,good
4,edu,95107,good


In [65]:
df_tld_bad_res = pd.DataFrame(df_bad.groupby("tld").count().sort_values("domain", ascending=False))
df_tld_bad_res = df_tld_bad_res.reset_index()
df_tld_bad_res["label"] = "bad"
df_tld_bad_res.head()

,tld,domain,label
0,com,243598,bad
1,net,25326,bad
2,uk,21954,bad
3,org,17946,bad
4,de,14533,bad


In [66]:
df_tld_prices = pd.concat([
    df_tld_good_res.merge(df_prices, on='tld', how='left'),
    df_tld_bad_res.merge(df_prices, on='tld', how='left')
])

In [67]:
df_tld_prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3023 entries, 0 to 1794
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tld                 3023 non-null   object 
 1   domain              3023 non-null   int64  
 2   label               3023 non-null   object 
 3   Registration Price  609 non-null    float64
 4   Renewal Price       609 non-null    float64
 5   TLD                 609 non-null    object 
 6   Minimum Term        609 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 188.9+ KB


In [68]:
df_tld_prices.head(20)

,tld,domain,label,Registration Price,Renewal Price,TLD,Minimum Term
0,com,1604974,good,8.99,13.99,.com,1 Year
1,org,265271,good,8.99,14.99,.org,1 Year
2,net,115300,good,12.99,15.99,.net,1 Year
3,uk,97508,good,9.99,9.99,.uk,1 Year
4,edu,95107,good,NaN,NaN,NaN,NaN
5,pl,68236,good,9.99,20.99,.pl,1 Year
6,ca,39348,good,12.99,12.99,.ca,1 Year
7,ru,38040,good,28.99,28.99,.ru,1 Year
8,au,32342,good,9.99,11.99,.au,1 Year
9,de,19176,good,NaN,NaN,NaN,NaN


In [69]:
df_tld_prices_clean = df_tld_prices.dropna().reset_index(drop=True)
df_tld_prices_clean.head()

,tld,domain,label,Registration Price,Renewal Price,TLD,Minimum Term
0,com,1604974,good,8.99,13.99,.com,1 Year
1,org,265271,good,8.99,14.99,.org,1 Year
2,net,115300,good,12.99,15.99,.net,1 Year
3,uk,97508,good,9.99,9.99,.uk,1 Year
4,pl,68236,good,9.99,20.99,.pl,1 Year


In [70]:
df_tld_prices_clean["Full registration price"] = df_tld_prices_clean["domain"] * df_tld_prices_clean["Registration Price"]
df_tld_prices_clean["Full renewal price"] = df_tld_prices_clean["domain"] * df_tld_prices_clean["Renewal Price"]
df_tld_prices_clean.head()

,tld,domain,label,Registration Price,Renewal Price,TLD,Minimum Term,Full registration price,Full renewal price
0,com,1604974,good,8.99,13.99,.com,1 Year,14428716.26,22453586.26
1,org,265271,good,8.99,14.99,.org,1 Year,2384786.29,3976412.29
2,net,115300,good,12.99,15.99,.net,1 Year,1497747.00,1843647.00
3,uk,97508,good,9.99,9.99,.uk,1 Year,974104.92,974104.92
4,pl,68236,good,9.99,20.99,.pl,1 Year,681677.64,1432273.64


In [71]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] =  110
plt.style.use('ggplot')

n = 10
name_intervals = 1

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
good_plot = df_tld_prices_clean[df_tld_prices_clean.label == "good"][:n]
bad_plot = df_tld_prices_clean[df_tld_prices_clean.label == "bad"][:n]

x_values = np.arange(n)
x_ticks = np.arange(0, n, name_intervals)

ax1.bar(x_values, good_plot["domain"], width=0.7, color="g")
ax1.set_xticklabels(good_plot.tld)
ax1.set_xticks(x_ticks)
ax1.title.set_text("Good")
ax1.set_ylabel("TLD frequency")
ax1.grid(False, axis="x")

ax2.bar(x_values, bad_plot["domain"], width=0.7)
ax2.set_xticklabels(bad_plot.tld)
ax2.set_xticks(x_ticks)
ax2.title.set_text("Bad")
ax2.grid(False, axis="x")

ax3.bar(x_values, good_plot["Registration Price"], width=0.7,  color="g")
ax3.set_xticklabels(good_plot.tld)
ax3.set_xticks(x_ticks)
ax3.set_ylabel("Price of the TLD in USD")
ax3.grid(False, axis="x")

ax4.bar(x_values, bad_plot["Registration Price"], width=0.7)
ax4.set_xticklabels(bad_plot.tld)
ax4.set_xticks(x_ticks)
ax4.grid(False, axis="x")

# fig.suptitle("Summary prices of the TLDs in $")

plt.subplots_adjust(wspace=0.25)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-71-d55d7b7f5921>:16: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(good_plot.tld)
<ipython-input-71-d55d7b7f5921>:23: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax2.set_xticklabels(bad_plot.tld)
<ipython-input-71-d55d7b7f5921>:29: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax3.set_xticklabels(good_plot.tld)
<ipython-input-71-d55d7b7f5921>:35: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax4.set_xticklabels(bad_plot.tld)


In [72]:
good_pr_data = df_tld_prices_clean[df_tld_prices_clean.label == "good"]
print("Mean for good domains:")
good_pr_data["Full renewal price"].sum() / good_pr_data["domain"].sum()

Mean for good domains:


16.422256197796617

In [73]:
bad_pr_data = df_tld_prices_clean[df_tld_prices_clean.label == "bad"]
print("Mean for bad domains:")
bad_pr_data["Full renewal price"].sum() / bad_pr_data["domain"].sum()

Mean for bad domains:


17.791481720140066

In [74]:
df_tld_prices_clean.sort_values("Registration Price")

,tld,domain,label,Registration Price,Renewal Price,TLD,Minimum Term,Full registration price,Full renewal price
436,live,208,bad,2.99,27.99,.live,1 Year,621.92,5821.92
521,download,4,bad,2.99,4.99,.download,1 Year,11.96,19.96
48,live,776,good,2.99,27.99,.live,1 Year,2320.24,21720.24
506,monster,7,bad,2.99,11.99,.monster,1 Year,20.93,83.93
494,date,10,bad,2.99,4.99,.date,1 Year,29.90,49.90
...,...,...,...,...,...,...,...,...,...
337,tickets,2,good,455.99,455.99,.tickets,1 Year,911.98,911.98
336,luxury,2,good,649.99,649.99,.luxury,1 Year,1299.98,1299.98
362,storage,1,good,650.99,650.99,.storage,1 Year,650.99,650.99
380,protection,1,good,2600.99,2600.99,.protection,1 Year,2600.99,2600.99


In [75]:
df_tld_prices_clean.groupby("label").count()

,tld,domain,Registration Price,Renewal Price,TLD,Minimum Term,Full registration price,Full renewal price
label,,,,,,,,
bad,218,218,218,218,218,218,218,218
good,391,391,391,391,391,391,391,391


In [76]:
df_tld_prices_clean.groupby("label").mean()

,domain,Registration Price,Renewal Price,Full registration price,Full renewal price
label,,,,,
bad,1836.633028,26.481101,39.802294,23273.377798,32676.422936
good,6220.138107,46.051535,60.627033,70661.931790,102148.701586


In [77]:
fig, ax = plt.subplots()


good_plot_2 = df_tld_prices_clean[np.logical_and(df_tld_prices_clean.label == "good", df_tld_prices_clean["Registration Price"] <= 150)]
bad_plot_2 = df_tld_prices_clean[np.logical_and(df_tld_prices_clean.label == "bad", df_tld_prices_clean["Registration Price"] <= 150)]
good_size = good_plot_2["domain"]/1000
bad_size = bad_plot_2["domain"]/300

ax.scatter(good_plot_2["Registration Price"], 
           good_plot_2["Renewal Price"], 
           c="g", s=good_size, alpha=0.5,
           label="good"
          )
ax.scatter(bad_plot_2["Registration Price"], 
           bad_plot_2["Renewal Price"], c="r", 
           s=bad_size, alpha=0.5,
           label="bad"
          )

ax.set_xlabel("Registration Price of the TLD in USD")
ax.set_ylabel("Renewal Price of the TLD in USD")
ax.title.set_text("The distribution of the TLD prices")
ax.legend(markerscale=0.5)

plt.subplots_adjust(wspace=0.25)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
df_tld_prices_clean[df_tld_prices_clean.label == "good"].sort_values("domain")

,tld,domain,label,Registration Price,Renewal Price,TLD,Minimum Term,Full registration price,Full renewal price
390,observer,1,good,10.99,10.99,.observer,1 Year,10.99,10.99
369,maison,1,good,58.99,58.99,.maison,1 Year,58.99,58.99
368,voyage,1,good,42.99,55.99,.voyage,1 Year,42.99,55.99
367,vodka,1,good,33.99,33.99,.vodka,1 Year,33.99,33.99
366,villas,1,good,58.99,58.99,.villas,1 Year,58.99,58.99
...,...,...,...,...,...,...,...,...,...
4,pl,68236,good,9.99,20.99,.pl,1 Year,681677.64,1432273.64
3,uk,97508,good,9.99,9.99,.uk,1 Year,974104.92,974104.92
2,net,115300,good,12.99,15.99,.net,1 Year,1497747.00,1843647.00
1,org,265271,good,8.99,14.99,.org,1 Year,2384786.29,3976412.29
